In [1]:
import openai
import pandas as pd
import tqdm
import glob

import warnings
warnings.filterwarnings("ignore")

In [2]:
with open("../apikey/apikey.txt", "r") as f:
    openai.api_key = f.readline().replace("\n", "")

In [3]:
def query_prompt(prompt, max_tokens=1000):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=max_tokens,
        temperature=0,
    )
    return response["choices"][0]["message"]["content"]

In [11]:
def get_df(fname):
    def f(categories):
        for cat in categories:
            if "science" in cat:
                return True
            if "physics" in cat:
                return True
            if "chemistry" in cat:
                return True
        return False
    
    def text_preprocess(text):
        return text.replace("===", "\n").replace("==", "\n")

    df = pd.read_parquet(fname)
    df_science = df[df["categories"].apply(f)]
    df_science["text"] = "title: " + df_science["title"] + "\n" + df_science["text"].apply(text_preprocess)
    return df_science.sample(len(df_science)//10)

In [12]:
files = glob.glob("../data/wikipedia/a.parquet")

In [13]:
import time

In [14]:
import pickle

In [15]:
import json

In [16]:
from datetime import datetime as dt

In [17]:
import traceback 
batch_size = 1

def make_prompt(series):
    prompt = f"""
# 依頼
You are a professor at a science university and are creating a test for your students.
Using the given text, create a question in which you select one of the five options for the question text to select the most appropriate statement. Also, extract the portion of the text that provides the basis for your answer.
The output should be an array in json format, with "prompt" as the question text, "A," "B," "C," "D," and "E" as the choices, "answer" as the answer choice, and "basis" as the rationale.
Also, please create one question per text. In other words, the total number of questions created will be one.
# text
## text 1 
{series['text']}


# attention
Please create one question per text. So the total number of problems created will be 1.
"""
    return prompt

def f(series):
    if series["A"] != series["A"]:
        if type(series["choices"]) == dict:
            for key in ["A", "B", "C", "D", "E"]:
                series[key] = series["choices"][key]
        elif type(series["choices"] == list):
            for i, key in enumerate(["A", "B", "C", "D", "E"]):
                series[key] = series["choices"][i]
    return series
texts = []
now_date = dt.now().strftime("%Y%m%d%H%M%S")

for f in files:
    df_science = get_df(f)
    
    for i in tqdm.tqdm(range(len(df_science)), desc=f):
        try:
            series = df_science.iloc[i]
            prompt = make_prompt(series)
            text = query_prompt(prompt)
            text = json.loads(text)
            if type(text) == list:
                text = text[0]
            text["wiki_id"] = series["id"]
            text["original_text"] = series["text"]
            texts.append(text)
        except Exception as e:
            print(e)
            traceback.print_exc()
            print(text)
        if i % 100 == 0:
            df_texts = pd.DataFrame(texts)
            df_texts = df_texts.apply(f, axis=1)

            df_texts.to_csv(f"output_gpt3.5_generate/{now_date}.csv")

../data/wikipedia/a.parquet:   8%|█████▍                                                               | 22/277 [01:54<26:19,  6.19s/it]Traceback (most recent call last):
  File "/tmp/ipykernel_23024/3350171404.py", line 30, in <module>
    text = query_prompt(prompt)
  File "/tmp/ipykernel_23024/2301454346.py", line 2, in query_prompt
    response = openai.ChatCompletion.create(
  File "/opt/conda/lib/python3.10/site-packages/openai/api_resources/chat_completion.py", line 25, in create
    return super().create(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/openai/api_resources/abstract/engine_api_resource.py", line 153, in create
    response, _, api_key = requestor.request(
  File "/opt/conda/lib/python3.10/site-packages/openai/api_requestor.py", line 298, in request
    resp, got_stream = self._interpret_response(result, stream)
  File "/opt/conda/lib/python3.10/site-packages/openai/api_requestor.py", line 700, in _interpret_response
    self._interpret_response_l

This model's maximum context length is 16385 tokens. However, your messages resulted in 16400 tokens. Please reduce the length of the messages.
{'prompt': 'Which of the following statements is supported by the given text?', 'choices': [{'label': 'A', 'text': 'The action potential pulse model takes into account entropy and the conduction speed of the action potential along an axon.'}, {'label': 'B', 'text': "The action potential pulse model is a continuation of Hodgkin Huxley's work in 1952."}, {'label': 'C', 'text': 'The action potential pulse model replaces saltatory conduction in myelinated axons.'}, {'label': 'D', 'text': 'The action potential pulse model is based on cable theory and the capacitance potential of a membrane.'}, {'label': 'E', 'text': 'The action potential pulse model explains the speed of transmission through unmyelinated axons.'}], 'answer': 'A', 'basis': "The text states that the action potential pulse model takes into account entropy and the conduction speed of th

../data/wikipedia/a.parquet:  90%|████████████████████████████████████████████████████████████▉       | 248/277 [19:43<02:12,  4.58s/it]Traceback (most recent call last):
  File "/tmp/ipykernel_23024/3350171404.py", line 30, in <module>
    text = query_prompt(prompt)
  File "/tmp/ipykernel_23024/2301454346.py", line 2, in query_prompt
    response = openai.ChatCompletion.create(
  File "/opt/conda/lib/python3.10/site-packages/openai/api_resources/chat_completion.py", line 25, in create
    return super().create(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/openai/api_resources/abstract/engine_api_resource.py", line 153, in create
    response, _, api_key = requestor.request(
  File "/opt/conda/lib/python3.10/site-packages/openai/api_requestor.py", line 298, in request
    resp, got_stream = self._interpret_response(result, stream)
  File "/opt/conda/lib/python3.10/site-packages/openai/api_requestor.py", line 700, in _interpret_response
    self._interpret_response_l

This model's maximum context length is 16385 tokens. However, your messages resulted in 18296 tokens. Please reduce the length of the messages.
{'prompt': 'What is the purpose of using high temperature and pressure in accelerated solvent extraction?', 'A': 'To decrease extraction efficiency', 'B': 'To increase extraction efficiency', 'C': 'To reduce the amount of solvent required', 'D': 'To decrease analyte recovery', 'E': 'To decrease the boiling point of the solvent', 'answer': 'B', 'basis': 'The text states that the elevated temperature is employed to increase extraction efficiency of the analyte of interest and the elevated pressure is used to keep the solvent in a liquid state as the temperature is increased above its boiling point.', 'wiki_id': '50484198', 'original_text': 'title: Accelerated solvent extraction\nAccelerated solvent extraction (ASE) is a method for extracting various chemicals from a complex solid or semisolid sample matrix. The process uses high temperature and p

../data/wikipedia/a.parquet: 100%|████████████████████████████████████████████████████████████████████| 277/277 [22:15<00:00,  4.82s/it]


In [40]:
df_texts.to_csv(f"output_gpt3.5_generate/{now_date}.csv")

In [20]:
pd.DataFrame(texts)["A"].isnull().sum()

69

In [41]:
df_science = get_df(files[0])
df_science[df_science["id"] == '29637793']

,id,title,text,categories


In [42]:
df_science

,id,title,text,categories
368626,1908395,Artificial brain,title: Artificial brain\nAn artificial brain (...,"[Computational neuroscience, Robotics, Emergin..."
34268,10410698,Abdul Amir al-Jamri,title: Abdul Amir al-Jamri\nSheikh Abdul Amir ...,"[1938 births, 2006 deaths, Deaths from kidney ..."
233077,1958222,Amyloid beta,title: Amyloid beta\nAmyloid beta (Aβ or Abeta...,"[Peptides, Molecular neuroscience, Alzheimer's..."
9870,3621668,A Woman of the Iron People,title: A Woman of the Iron People\nA Woman of ...,"[1991 American novels, 1991 science fiction no..."
139719,38366604,Albert Spaier,title: Albert Spaier\nAlbert Spaier (9 July 18...,"[1883 births, 1934 deaths, Writers from Iași, ..."
...,...,...,...,...
62685,4474244,Actuarial reserves,"title: Actuarial reserves\nIn insurance, an ac...","[Actuarial science, Capital requirement de:Dec..."
357456,4260564,Arrival II,title: Arrival II\nArrival ll (alternatively t...,"[1998 films, 1998 science fiction films, Ameri..."
391514,32894329,Astrobiophysics,title: Astrobiophysics\nAstrobiophysics is a f...,"[Astrophysics, Biophysics .]"
10000,2824171,A World of Difference (novel),title: A World of Difference (novel)\nA World ...,"[1990 American novels, Novels set during the C..."
